# Регрессионный анализ в полном факторном эксперименте (ПФЭ 2³)

## Задача
Исследуем влияние трёх факторов на **предел текучести стали** (МПа):
- $ X_1 $: содержание углерода (0.1% – 0.4%)
- $ X_2 $: толщина материала (1.0 мм – 2.0 мм)
- $ X_3 $: угол относительно направления проката (0° – 90°)

Проведён **ПФЭ 2³** (8 опытов), каждый опыт повторён **3 раза**.

Модель:  
$$
\hat{y} = b_0 + b_1 x_1 + b_2 x_2 + b_3 x_3
$$

Все расчёты — вручную, с проверкой:
- Однородности дисперсий (критерий Кохрена)
- Значимости коэффициентов (критерий Стьюдента)
- Адекватности модели (критерий Фишера)

In [ ]:
import numpy as np
from itertools import product

# Настройки отображения
np.set_printoptions(precision=4, suppress=True)

## 1. Матрица планирования (кодированные факторы)

Кодировка:  
- низкий уровень → -1  
- высокий уровень → +1

In [ ]:
# Все комбинации уровней
factors = list(product([-1, 1], repeat=3))
X_coded = np.array(factors)  # (8, 3)

print("Матрица планирования (x1, x2, x3):")
print(X_coded)

## 2. Экспериментальные данные с повторениями

Каждый из 8 опытов повторён 3 раза.

In [ ]:
# Отклики: каждая строка — 3 измерения одного опыта
y_repeated = np.array([
    [320, 325, 318],  # Опыт 1
    [340, 338, 342],  # Опыт 2
    [310, 312, 309],  # Опыт 3
    [350, 352, 348],  # Опыт 4
    [300, 298, 302],  # Опыт 5
    [330, 332, 329],  # Опыт 6
    [290, 288, 292],  # Опыт 7
    [360, 362, 358],  # Опыт 8
])

print("Отклики с повторениями:")
print(y_repeated)

## 3. Средние и выборочные дисперсии

Для каждого опыта:
- $ \bar{y}_i = \frac{1}{m} \sum y_{ij} $
- $ S_i^2 = \frac{1}{m-1} \sum (y_{ij} - \bar{y}_i)^2 $

In [ ]:
m = y_repeated.shape[1]  # число повторений = 3
n = y_repeated.shape[0]  # число опытов = 8

y_mean = np.mean(y_repeated, axis=1)  # (8,)
S2 = np.var(y_repeated, axis=1, ddof=1)  # (8,)

print("Опыт | Среднее | Дисперсия")
print("-----|---------|----------")
for i in range(n):
    print(f" {i+1:2}  |  {y_mean[i]:5.1f}  |  {S2[i]:6.3f}")

## 4. Проверка однородности дисперсий по критерию Кохрена

$$
G = \frac{\max(S_i^2)}{\sum_{i=1}^n S_i^2}
$$

Сравниваем с табличным значением $ G_{\text{крит}}(q=0.05,\ f=m-1=2,\ n=8) \approx 0.522 $

In [ ]:
G_observed = np.max(S2) / np.sum(S2)
G_critical = 0.522  # табличное значение

print(f"Наблюдаемое G: {G_observed:.4f}")
print(f"Критическое G: {G_critical}")

if G_observed < G_critical:
    print("✅ Дисперсии однородны. Можно продолжать анализ.")
else:
    print("❌ Дисперсии неоднородны. Модель может быть ненадёжной.")

## 5. Средняя дисперсия воспроизводимости

$$
S^2_{\text{воспр}} = \frac{1}{n} \sum_{i=1}^n S_i^2
$$

In [ ]:
S2_vospr = np.mean(S2)
print(f"Средняя дисперсия воспроизводимости: {S2_vospr:.3f}")

## 6. Построение модели МНК

Матрица плана с константой:
$$
\mathbf{X} = [\mathbf{1},\ \mathbf{x}_1,\ \mathbf{x}_2,\ \mathbf{x}_3]
$$

Коэффициенты:
$$
\mathbf{b} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

In [ ]:
X = np.column_stack([np.ones(n), X_coded])  # (8, 4)
y = y_mean  # вектор средних

# МНК
XTX = X.T @ X
XTX_inv = np.linalg.inv(XTX)
b = XTX_inv @ X.T @ y

print("Коэффициенты модели:")
print(f"b0: {b[0]:.2f}")
print(f"b1: {b[1]:.2f}")
print(f"b2: {b[2]:.2f}")
print(f"b3: {b[3]:.2f}")

## 7. Проверка значимости коэффициентов по критерию Стьюдента

Стандартная ошибка:
$$
\text{SE}(b_j) = \sqrt{\frac{S^2_{\text{воспр}}}{N}}, \quad N = 8
$$

t-статистика:
$$
t_j = \frac{|b_j|}{\text{SE}(b_j)}
$$

Сравниваем с $ t_{\text{крит}}(f=16,\ \alpha=0.05) = 2.120 $

In [ ]:
SE_b = np.sqrt(S2_vospr / n)  # ошибка коэффициента
t_critical = 2.120  # t_{0.025}(16)
t_values = np.abs(b) / SE_b

print(f"Стандартная ошибка: {SE_b:.3f}")
print(f"Критическое t: {t_critical}")
print("\nКоэффи. | t_набл | Значимость")
print("--------|--------|-----------")
for j, name in enumerate(['b0', 'b1 (C)', 'b2 (h)', 'b3 (θ)']):
    t_j = t_values[j]
    sig = "Да" if t_j > t_critical else "Нет"
    print(f"{name:7} | {t_j:6.3f} | {sig}")

## 8. Проверка адекватности по критерию Фишера

- Предсказания: $ \hat{y} = \mathbf{X} \mathbf{b} $
- Остаточная сумма квадратов: $ SS_{\text{ост}} = \sum (y_i - \hat{y}_i)^2 $
- Степени свободы: $ f_{\text{ост}} = n - p = 8 - 4 = 4 $
- Дисперсия адекватности: $ S^2_{\text{ад}} = SS_{\text{ост}} / f_{\text{ост}} $
- $ F = S^2_{\text{ад}} / S^2_{\text{воспр}} $
- Сравниваем с $ F_{\text{крит}}(4, 2) = 19.25 $

In [ ]:
y_pred = X @ b
residuals = y - y_pred
SS_ost = np.sum(residuals**2)
f_ost = n - len(b)  # 8 - 4 = 4
S2_adeq = SS_ost / f_ost

F_observed = S2_adeq / S2_vospr
F_critical = 19.25  # F(4, 2) при α=0.05

print(f"Остаточная сумма квадратов: {SS_ost:.2f}")
print(f"Дисперсия адекватности: {S2_adeq:.3f}")
print(f"Наблюдаемое F: {F_observed:.3f}")
print(f"Критическое F (f1=4, f2=2): {F_critical}")

if F_observed < F_critical:
    print("✅ Модель адекватна.")
else:
    print("❌ Модель неадекватна. Рассмотрите взаимодействия.")

## 9. Итоговая модель

$$
\hat{y} = {b[0]:.1f} + {b[1]:.1f} x_1 + {b[2]:.1f} x_2 + {b[3]:.1f} x_3
$$

Где:
- $ x_1 = \frac{C - 0.25}{0.15} $
- $ x_2 = \frac{h - 1.5}{0.5} $
- $ x_3 = \frac{\theta - 45}{45} $

Модель:
- построена на однородных данных (Кохрен ✅)
- коэффициенты $ b_1 $ и $ b_3 $ — значимы
- модель адекватна (Фишер ✅)

## ✅ Выводы

- Все статистические проверки выполнены вручную.
- Модель пригодна для прогнозирования предела текучести.
- При необходимости можно упростить модель, исключив незначимые члены.
- Для улучшения точности — добавить взаимодействия или перейти к дробному факторному эксперименту.

> 💡 Рекомендация: всегда проводите повторения опытов при планировании эксперимента!